In [1]:
import toytree
import ipcoal
import ipyrad.analysis as ipa

In [2]:
# Load a RAxML tree.  This tree was built from subsampling scaffold 05 of the assembly,
# with 217/261 taxa and 5% missing data.
tre = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.pt_scaffold05")

In [3]:
# Divide RAxML branch lengths (expected subs per site) by mutation rate (expected subs per site per gen) to get generations
# for branches.
tre2 = tre.set_node_values(
    "dist",
    {i.name: i.dist / 1e-8 for i in tre.get_feature_dict()}
)

In [4]:
# Draw tree to be used for ipcoal simulations.
tre2.draw(
    ts='p',
    edge_type='p',
    tip_labels_align=True,
    tip_labels=False,
    node_labels=False,
    node_sizes=0,
    width=800,
    height=600,
);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t4bf348df977148bba513231858da2963" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 600.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 383483 766967 1150450 1533934 1917417

In [ ]:
# Simualte sequence data on coalescent genealogies, write results to Nexus file.
model = ipcoal.Model(tre2, Ne=1e5, nsamples=2, seed=123)
model.sim_loci(nloci=100, nsites=200)
model.write_concat_to_nexus(name="mbtest-a", outdir='/tmp', diploid=True)

In [ ]:
# Initialize mb object for test under relaxed clock model.
mba = ipa.mrbayes(
    data="/tmp/mbtest-a.nex,
    name="itest-a",
    workdir="/moto/eaton/users/hnl2109/analysis-mrbayes/",
    clock_model=2,
    constraints=tre2,
    ngen=int(1e6),
    nruns=2,
)

# Print parameters.
print(mba.params)